# Convert Osu to Beet Saber

In [14]:
# find install dir
import os
import zipfile

In [11]:
osz_path = '/Users/donu/Desktop/S25/ELEC 327/327-final-proj/oszs/'
unzip_path = '/Users/donu/Desktop/S25/ELEC 327/327-final-proj/oszs/charts/'

In [16]:
# find all osu zips
oszs = [osz_path+x for x in os.listdir(osz_path) if x.endswith('osz')]
oszs

['/Users/donu/Desktop/S25/ELEC 327/327-final-proj/oszs/killing-my-love.osz',
 '/Users/donu/Desktop/S25/ELEC 327/327-final-proj/oszs/duvet.osz',
 '/Users/donu/Desktop/S25/ELEC 327/327-final-proj/oszs/running-man.osz']

In [20]:
def retain_file(filepath):
  """Returns if file should be kept
  At time of writing, keeps osu metadata files and mp3 files
  (wav files are used only for hitsounds (irrelevant))"""
  if not (filepath.endswith('osu') or filepath.endswith('mp3')):
    return False
  return True

In [55]:
def keep_file(filepath):
  """
  Tells which files should be kept
  """
  checks = ['easy' in filepath.lower(),
            'normal' in filepath.lower(),
            'hard' in filepath.lower(),
            'insane' in filepath.lower(),
            'extra' in filepath.lower(),
            'expert' in filepath.lower(),
            'audio.mp3' in filepath,
            ]
  
  return ((True in checks) and ('.wav' not in filepath))

In [56]:
# unzip them. how?
# # make dir per song
# # retain metadata and audio
os.makedirs(unzip_path, exist_ok=True)
for osz in oszs:
  songname = osz.split('/')[-1].split('.')[0]
  save_path = unzip_path+songname+'/'
  with zipfile.ZipFile(osz, 'r') as zip_ref:
    zip_ref.extractall(save_path)
# delete unneeded files:
subdirs = [entry for entry in os.listdir(unzip_path) if os.path.isdir(os.path.join(unzip_path, entry))]
for subdir in subdirs:
  dirpath = os.path.join(unzip_path,subdir)
  for file in os.listdir(dirpath):
    if not keep_file(file):
      os.remove(os.path.join(dirpath,file))